In [1]:
import pandas as pd
import snowflake.connector
from datetime import datetime, timedelta
import boto3
from dotenv import load_dotenv
import os



ModuleNotFoundError: No module named 'dotenv'

In [ ]:
load_dotenv()

# Access the AWS credentials using the environment variables
access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")

In [ ]:
conn = snowflake.connector.connect(
    user='SANJAYKASHYAP',
    password='Bigdata@23',
    account='iogoldm-vcb38713',
    warehouse='COMPUTE_WH',
    database='SEVIR_META',
    schema='noes'
)



In [ ]:

def check_last_updated_date_from_snowflake():
    # Create a cursor object
    cur = conn.cursor()
    query = """SELECT
                "year" AS latest_year,
                MAX("month") AS latest_month,
                MAX("day") AS latest_day
            FROM SEVIR_META.PUBLIC.NOES
            WHERE "year" = (
                SELECT MAX("year") FROM SEVIR_META.PUBLIC.NOES
            )
            GROUP BY "year";"""
    # Execute the SELECT statement to get the last N records from the table
    cur.execute(query)

    # Fetch the results as a list of tuples
    results = cur.fetchall()

    # Close the cursor and the database connection
    cur.close()
    conn.close()
    t = results[0]
    last_updated = datetime(t[0], 1, 1) + timedelta(t[1] - 1) + timedelta(hours=t[2])
    return last_updated


In [ ]:


def connect_to_goes_s3():
    # Define the S3 bucket and prefix
    bucket_name = "noaa-goes18"
    prefix = "ABI-L1b-RadC/"
    s3 = boto3.resource("s3")
    return s3, bucket_name, prefix




In [ ]:
def get_metadata_and_store(s3, bucket_name, prefix, last_updated):
    names = []
    paginator = s3.meta.client.get_paginator("list_objects_v2")
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

    # Loop through each page of objects
    for page in page_iterator:
        # Loop through each object in the page
        for obj in page.get("Contents", []):
            key = obj.get("Key")
            if key.endswith(".nc"):
                # Parse the file name to get the year and day of year
                parts = key.split("/")
                t = (int(parts[1]), int(parts[2]),int(parts[3]))
                file_date = datetime(t[0], 1, 1) + timedelta(t[1] - 1) + timedelta(hours=t[2])
                # Check if the file is newer than the last updated date in Snowflake
                if file_date > last_updated:
                    names.append(key)
    return names



In [ ]:
def metadata_data_frame(names):
    data = pd.DataFrame([i.split('/') for i in names], columns=['Product Name','Year','Day','Hour','File Name'])
    data.drop('Product Name', axis=1, inplace=True)
    data.drop('File Name', axis=1, inplace=True)
    data['datetime'] = pd.to_datetime(data['Year'] + ' ' + data['Day'], format='%Y %d')
    data['year'] = data['datetime'].dt.year
    data['day_of_year'] = data['datetime'].dt.dayofyear
    data.drop('datetime', axis=1, inplace=True)
    return data



In [ ]:

last_updated = check_last_updated_date_from_snowflake()


DatabaseError: 250002 (08003): Connection is closed

In [ ]:
print(last_updated)

2023-02-21 00:00:00


In [ ]:
s3, bucket_name, prefix = connect_to_goes_s3()


In [ ]:
names = get_metadata_and_store(s3, bucket_name, prefix, last_updated)


In [ ]:
print(names)[0]

In [ ]:
data = metadata_data_frame(names)
print(data[0])

ValueError: unconverted data remains: 2